In [ ]:
# Preamble 
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
    %config Completer.use_jedi = False
except:
    pass

import numpy as np
from astropy.table import Table, join
from astropy.io import fits
import time
import matplotlib.pyplot as plt
from scipy import signal, optimize
import glob

In [ ]:
reduction_download_date = 220714
reduction_download_date = 220902
reduction_download_date = 230101

In [ ]:
dr60 = Table.read('../observations/dr6.0_'+str(reduction_download_date)+'.fits')
# dr60.sort('sobject_id')
# uni, uni_ind = np.unique(dr60['sobject_id'],return_index=True)
# if len(dr60['sobject_id']) != len(uni):
#     dr60 = dr60[uni_ind]
#     dr60.write('../observations/dr6.0_220701.fits',overwrite=True)

In [ ]:
try:
    extra_info = Table.read('../observations/dr6.0_'+str(reduction_download_date)+'_additional_wavelength_info.fits')
except:
    extra_info = Table()
    extra_info['sobject_id'] = dr60['sobject_id']
    extra_info['date'] = np.array([str(x)[:6] for x in dr60['sobject_id']],dtype=str)
    extra_info['SLITMASK'] = np.zeros(len(dr60['sobject_id']),dtype=int); extra_info['SLITMASK'][:] = 2

    for key in ['CRVAL1', 'CDELT1', 'RVCOM', 'RV']:
        for ccd in [1,2,3,4]:
            extra_info[key+'_ccd'+str(ccd)] = np.zeros(len(dr60['sobject_id']),dtype=np.float32); extra_info[key+'_ccd'+str(ccd)][:] = np.NaN
            
    start_time = time.time()

    for index, sobject_id in enumerate(dr60['sobject_id']):

        if index%1000 == 1:
            time_now = time.time()
            print(index,'/',len(dr60['sobject_id']))
            print('took ',time_now-start_time,' s, expected finish: ', (time_now-start_time)/index * (len(dr60['sobject_id']) - index))

        for ccd in [1,2,3,4]:
            try:
                fits_file = fits.open('../observations/'+str(sobject_id)[:6]+'/spectra/com/'+str(sobject_id)+str(ccd)+'.fits')

                if ccd == 1:
                    if fits_file[0].header['SLITMASK'] == 'OUT':
                        extra_info['SLITMASK'][index] = 0
                    elif fits_file[0].header['SLITMASK'] == 'IN':
                        extra_info['SLITMASK'][index] = 1
                    else:
                        extra_info['SLITMASK'][index] = 3

                for key in ['CRVAL1', 'CDELT1', 'RVCOM', 'RV']:
                    try:
                        extra_info[key+'_ccd'+str(ccd)][index] = fits_file[0].header[key]
                    except:
                        pass

                fits_file.close()
            except:
                pass
    #             print('Failed for '+str(sobject_id)+' at index '+str(index),' for ',key)
    
    
    # Get rid of precision errors in data formats
    extra_info['rvcom_d3'] = np.round(dr60['rv_com'],decimals=2)
    extra_info['RVCOM_d3_ccd1'] = np.round(extra_info['RVCOM_ccd1'],decimals=2)
    extra_info['RVCOM_d3_ccd2'] = np.round(extra_info['RVCOM_ccd2'],decimals=2)
    extra_info['RVCOM_d3_ccd3'] = np.round(extra_info['RVCOM_ccd3'],decimals=2)
    extra_info['RVCOM_d3_ccd4'] = np.round(extra_info['RVCOM_ccd4'],decimals=2)
    
    extra_info['RVCOM_same_in_tables'] = extra_info['rvcom_d3'] == extra_info['RVCOM_d3_ccd1']
    extra_info['RVCOM_d3_ccd_same'] = np.all([extra_info['RVCOM_d3_ccd1'] == extra_info['RVCOM_d3_ccd2'],extra_info['RVCOM_d3_ccd1'] == extra_info['RVCOM_d3_ccd3'],extra_info['RVCOM_d3_ccd1'] == extra_info['RVCOM_d3_ccd4']],axis=0)
    
    extra_info.write('../observations/dr6.0_'+str(reduction_download_date)+'_additional_wavelength_info.fits',overwrite=True)
    

In [ ]:
ranges = dict()

f, gs = plt.subplots(2,4,figsize=(15,7.5),sharey=True)

for ccd in [1,2,3,4]:
    for cindex, key in enumerate(['CRVAL1','CDELT1']):
        
        ax = gs[cindex,ccd-1]
        
        ranges[key+'_ccd'+str(ccd)+'_p1'], ranges[key+'_ccd'+str(ccd)+'_p99'] = np.nanpercentile(extra_info[key+'_ccd'+str(ccd)],q=[0.1,99.9])
        if key == 'CRVAL1':
            if ccd == 1:
                ranges[key+'_ccd'+str(ccd)+'_p1'] = 4711.5
                ranges[key+'_ccd'+str(ccd)+'_p99'] = 4715.5
            if ccd == 2:
                ranges[key+'_ccd'+str(ccd)+'_p1'] = 5646.7
                ranges[key+'_ccd'+str(ccd)+'_p99'] = 5650.7
            if ccd == 3:
                ranges[key+'_ccd'+str(ccd)+'_p1'] = 6475.8
                ranges[key+'_ccd'+str(ccd)+'_p99'] = 6480.5
            if ccd == 4:
                ranges[key+'_ccd'+str(ccd)+'_p1'] = 7583.5
                ranges[key+'_ccd'+str(ccd)+'_p99'] = 7587.5
        if key == 'CDELT1':
            if ccd == 1:
                ranges[key+'_ccd'+str(ccd)+'_p1'] = 0.04591
                ranges[key+'_ccd'+str(ccd)+'_p99'] = 0.04605
            if ccd == 2:
                ranges[key+'_ccd'+str(ccd)+'_p1'] = 0.0545
                ranges[key+'_ccd'+str(ccd)+'_p99'] = 0.0548
            if ccd == 3:
                ranges[key+'_ccd'+str(ccd)+'_p1'] = 0.06303
                ranges[key+'_ccd'+str(ccd)+'_p99'] = 0.063215
            if ccd == 4:
                ranges[key+'_ccd'+str(ccd)+'_p1'] = 0.07342
                ranges[key+'_ccd'+str(ccd)+'_p99'] = 0.07367
                
        left = ranges[key+'_ccd'+str(ccd)+'_p1'] - 3*(ranges[key+'_ccd'+str(ccd)+'_p99'] - ranges[key+'_ccd'+str(ccd)+'_p1'])
        right = ranges[key+'_ccd'+str(ccd)+'_p99'] + 3*(ranges[key+'_ccd'+str(ccd)+'_p99'] - ranges[key+'_ccd'+str(ccd)+'_p1'])
        ax.hist(
            extra_info[key+'_ccd'+str(ccd)],
            bins = np.linspace(left,right,200),
            log = True
        )
        ax.axvline(ranges[key+'_ccd'+str(ccd)+'_p1'],c='r')
        ax.axvline(ranges[key+'_ccd'+str(ccd)+'_p99'],c='r')
        ax.set_xlabel(key+'_ccd'+str(ccd))
        
        inside = (extra_info[key+'_ccd'+str(ccd)] > ranges[key+'_ccd'+str(ccd)+'_p1']) & (extra_info[key+'_ccd'+str(ccd)] < ranges[key+'_ccd'+str(ccd)+'_p99'])
        
        ax.text(0.05,0.95,'Inside: '+str(len(extra_info[key+'_ccd'+str(ccd)][inside]))+' ('+"{:.1f}".format(100.*len(extra_info[key+'_ccd'+str(ccd)][inside])/len(extra_info[key+'_ccd'+str(ccd)]))+'%)',transform=ax.transAxes)
        ax.text(0.05,0.875,'Outside: '+str(len(extra_info[key+'_ccd'+str(ccd)][~inside]))+' ('+"{:.1f}".format(100.*len(extra_info[key+'_ccd'+str(ccd)][~inside])/len(extra_info[key+'_ccd'+str(ccd)]))+'%)',transform=ax.transAxes)
plt.tight_layout()
plt.savefig('figures/galah_dr4_crval_cdelt_histograms.png',dpi=200,bbox_inches='tight')
plt.show()
plt.close()


In [ ]:
print('ccd','date','len(date)','len(date+outside)','len(date+outside+flagged)')
for ccd in [1,2,3,4]:
    
    inside = (
        (extra_info['CRVAL1_ccd'+str(ccd)] > ranges['CRVAL1_ccd'+str(ccd)+'_p1']) & (extra_info['CRVAL1_ccd'+str(ccd)] < ranges['CRVAL1_ccd'+str(ccd)+'_p99']) &
        (extra_info['CDELT1_ccd'+str(ccd)] > ranges['CDELT1_ccd'+str(ccd)+'_p1']) & (extra_info['CDELT1_ccd'+str(ccd)] < ranges['CDELT1_ccd'+str(ccd)+'_p99'])
    )
    outside = ~inside

    for date in np.unique(extra_info['date'][outside]):
        len_on_that_date = len(extra_info['date'][extra_info['date']==date])
        len_outside_on_that_date = len(extra_info['date'][outside & (extra_info['date']==date)])
        len_outside_on_that_date_flagged = len(extra_info['date'][outside & (extra_info['date']==date) & (dr60['reduction_flags']>0)])

        if (len_outside_on_that_date > 50) | (len_outside_on_that_date > 0.1*len_on_that_date):
            print(ccd, date,len_on_that_date,len_outside_on_that_date,len_outside_on_that_date_flagged)

In [ ]:
for ccd in [1,2,3,4]:
    
    inside = (
        (extra_info['CRVAL1_ccd'+str(ccd)] > ranges['CRVAL1_ccd'+str(ccd)+'_p1']) & (extra_info['CRVAL1_ccd'+str(ccd)] < ranges['CRVAL1_ccd'+str(ccd)+'_p99']) &
        (extra_info['CDELT1_ccd'+str(ccd)] > ranges['CDELT1_ccd'+str(ccd)+'_p1']) & (extra_info['CDELT1_ccd'+str(ccd)] < ranges['CDELT1_ccd'+str(ccd)+'_p99'])
    )
    outside = ~inside

    for pivot in np.unique(dr60['pivot'][outside]):
        len_on_that_pivot = len(dr60['pivot'][dr60['pivot']==pivot])
        len_outside_on_that_pivot = len(dr60['pivot'][outside & (dr60['pivot']==pivot)])
        len_outside_on_that_pivot_flagged = len(dr60['pivot'][outside & (dr60['pivot']==pivot) & (dr60['reduction_flags']>0)])

        if (len_outside_on_that_pivot > 50) | (len_outside_on_that_pivot > 0.1*len_on_that_pivot):
            print(ccd, pivot,len_on_that_pivot,len_outside_on_that_pivot,len_outside_on_that_pivot_flagged)

In [ ]:
for date in np.unique(extra_info['date'])[:2]:
    
    f, gs = plt.subplots(4,2,figsize=(7.5,10))
    
    observations_of_that_night = date == extra_info['date']
    
    for ccd in [1,2,3,4]:

        ax = gs[ccd-1,0]
        ax.set_title(date)
        
        observations_of_that_night_without_reduction_flag = observations_of_that_night & (dr60['reduction_flags'] == 0)
        observations_of_that_night_with_reduction_flag = observations_of_that_night & (dr60['reduction_flags'] != 0)

        ax.scatter(
            extra_info['CRVAL1_ccd'+str(ccd)][observations_of_that_night_without_reduction_flag],
            extra_info['CDELT1_ccd'+str(ccd)][observations_of_that_night_without_reduction_flag],
            c = 'C0',
            s=1,
            label = 'red_flag=0'
        )
        ax.scatter(
            extra_info['CRVAL1_ccd'+str(ccd)][observations_of_that_night_with_reduction_flag],
            extra_info['CDELT1_ccd'+str(ccd)][observations_of_that_night_with_reduction_flag],
            c = 'orange',
            s=1,
            label = 'red_flag > 0'
        )
        
        ax.set_xlim(1.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p1'] - 0.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p99'], 1.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p99'] -0.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p1'])
        ax.set_ylim(1.05 * ranges['CDELT1_ccd'+str(ccd)+'_p1'] - 0.05 * ranges['CDELT1_ccd'+str(ccd)+'_p99'], 1.05 * ranges['CDELT1_ccd'+str(ccd)+'_p99'] -0.05 * ranges['CDELT1_ccd'+str(ccd)+'_p1'])

        left_outside = extra_info['CRVAL1_ccd'+str(ccd)] < 1.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p1'] - 0.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p99']
        right_outside = extra_info['CRVAL1_ccd'+str(ccd)] > 1.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p99'] -0.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p1']
        top_outside = extra_info['CDELT1_ccd'+str(ccd)] < 1.05 * ranges['CDELT1_ccd'+str(ccd)+'_p1'] - 0.05 * ranges['CDELT1_ccd'+str(ccd)+'_p99']
        bottom_outside = extra_info['CDELT1_ccd'+str(ccd)] > 1.05 * ranges['CDELT1_ccd'+str(ccd)+'_p99'] -0.05 * ranges['CDELT1_ccd'+str(ccd)+'_p1']
        
        outside_no_flag = observations_of_that_night_without_reduction_flag & (left_outside | right_outside | top_outside | bottom_outside)
        ax.scatter(
            extra_info['CRVAL1_ccd'+str(ccd)][outside_no_flag].clip(min=ranges['CRVAL1_ccd'+str(ccd)+'_p1'],max=ranges['CRVAL1_ccd'+str(ccd)+'_p99']),
            extra_info['CDELT1_ccd'+str(ccd)][outside_no_flag].clip(min=ranges['CDELT1_ccd'+str(ccd)+'_p1'],max=ranges['CDELT1_ccd'+str(ccd)+'_p99']),
            c = 'purple',
            marker = 'd',
            s = 2,
            label = 'red_flag = 0'
        )
        outside_flag = observations_of_that_night_with_reduction_flag & (left_outside | right_outside | top_outside | bottom_outside)
        ax.scatter(
            extra_info['CRVAL1_ccd'+str(ccd)][outside_no_flag].clip(min=ranges['CRVAL1_ccd'+str(ccd)+'_p1'],max=ranges['CRVAL1_ccd'+str(ccd)+'_p99']),
            extra_info['CDELT1_ccd'+str(ccd)][outside_no_flag].clip(min=ranges['CDELT1_ccd'+str(ccd)+'_p1'],max=ranges['CDELT1_ccd'+str(ccd)+'_p99']),
            c = 'r',
            marker = 'd',
            s = 2,
            label = 'red_flag > 0'
        )

        ax.set_xlabel('CRVAL1 CCD'+str(ccd))
        ax.set_ylabel('CDELT1 CCD'+str(ccd))

        if ccd == 1:
            ax.legend(ncol=2,fontsize=5)
            
        # Now Radial velocities
        ax = gs[ccd-1,1]
        ax.set_title(date)
        
        observations_of_that_night_with_same_rv = observations_of_that_night & extra_info['RVCOM_d3_ccd_same']
        observations_of_that_night_with_diff_rv = observations_of_that_night & (~extra_info['RVCOM_d3_ccd_same'])

        ax.scatter(
            extra_info['CRVAL1_ccd'+str(ccd)][observations_of_that_night_with_same_rv],
            extra_info['CDELT1_ccd'+str(ccd)][observations_of_that_night_with_same_rv],
            c = 'C0',
            s = 1,
            label = 'Same RVCOM'
        )
        ax.scatter(
            extra_info['CRVAL1_ccd'+str(ccd)][observations_of_that_night_with_diff_rv],
            extra_info['CDELT1_ccd'+str(ccd)][observations_of_that_night_with_diff_rv],
            c = 'r',
            s = 1,
            label = 'Diff RVCOM'
        )
        
        outside_no_flag = observations_of_that_night_with_same_rv & (left_outside | right_outside | top_outside | bottom_outside)
        ax.scatter(
            extra_info['CRVAL1_ccd'+str(ccd)][outside_no_flag].clip(min=ranges['CRVAL1_ccd'+str(ccd)+'_p1'],max=ranges['CRVAL1_ccd'+str(ccd)+'_p99']),
            extra_info['CDELT1_ccd'+str(ccd)][outside_no_flag].clip(min=ranges['CDELT1_ccd'+str(ccd)+'_p1'],max=ranges['CDELT1_ccd'+str(ccd)+'_p99']),
            c = 'purple',
            marker = 'd',
            s = 2,
            label = 'red_flag = 0'
        )
        outside_flag = observations_of_that_night_with_diff_rv & (left_outside | right_outside | top_outside | bottom_outside)
        ax.scatter(
            extra_info['CRVAL1_ccd'+str(ccd)][outside_no_flag].clip(min=ranges['CRVAL1_ccd'+str(ccd)+'_p1'],max=ranges['CRVAL1_ccd'+str(ccd)+'_p99']),
            extra_info['CDELT1_ccd'+str(ccd)][outside_no_flag].clip(min=ranges['CDELT1_ccd'+str(ccd)+'_p1'],max=ranges['CDELT1_ccd'+str(ccd)+'_p99']),
            c = 'r',
            marker = 'd',
            s = 2,
            label = 'red_flag > 0'
        )
        
        ax.set_xlim(1.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p1'] - 0.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p99'], 1.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p99'] -0.05 * ranges['CRVAL1_ccd'+str(ccd)+'_p1'])
        ax.set_ylim(1.05 * ranges['CDELT1_ccd'+str(ccd)+'_p1'] - 0.05 * ranges['CDELT1_ccd'+str(ccd)+'_p99'], 1.05 * ranges['CDELT1_ccd'+str(ccd)+'_p99'] -0.05 * ranges['CDELT1_ccd'+str(ccd)+'_p1'])
        ax.set_xlabel('CRVAL1 CCD'+str(ccd))
        ax.set_ylabel('CDELT1 CCD'+str(ccd))

        if ccd == 1:
            ax.legend(ncol=2,fontsize=5)

    plt.tight_layout()
    plt.savefig('wavelength_solution/galah_dr4_cdelt1_crval1_'+str(date)+'.png',dpi=200,bbox_inches='tight')
    plt.show()
    plt.close()

In [ ]:
bad_cdelt1_ccd2 = extra_info['CDELT1_ccd2'] < 0.0542
# This always seems to be CCD2!

In [ ]:
extra_info[(
    (dr60['pivot'] == 282) &
    np.isfinite(extra_info['RVCOM_d3_ccd2'])
)]